In [ ]:
import numpy
import scipy.io.wavfile
from scipy.fftpack import dct

import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display

In [ ]:
audio_file = 'audio/kita.wav'
y, sample_rate = librosa.load(audio_file, sr=None, mono=False) #file stereo
#sample_rate, signal = scipy.io.wavfile.read('audio/zero_2.wav') #file mono

signal = librosa.to_mono(y) # Convert stereo to mono

In [ ]:
# Waveform
pd.Series(signal).plot(figsize=(20, 5))
plt.show()

PRE-EMPHASIZE

In [ ]:
pre_emphasis = 0.97 #diantara 0.9 dan 0.98
emphasized_signal = numpy.append(signal[0], signal[1:] - pre_emphasis * signal[:-1]) #y(t)=x(t)−αx(t−1)

In [ ]:
pd.Series(emphasized_signal).plot(figsize=(20, 5))
plt.show()

FRAMING

In [ ]:
frame_size = 0.025
frame_stride = 0.01

In [ ]:
frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate
signal_length = len(emphasized_signal)
frame_length = int(round(frame_length))
frame_step = int(round(frame_step))
num_frames = int(numpy.ceil(float(numpy.abs(signal_length - frame_length)) / frame_step)) 

pad_signal_length = num_frames * frame_step + frame_length
z = numpy.zeros((pad_signal_length - signal_length))
pad_signal = numpy.append(emphasized_signal, z)

indices = numpy.tile(numpy.arange(0, frame_length), (num_frames, 1)) + numpy.tile(numpy.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
frames = pad_signal[indices.astype(numpy.int32, copy=False)]

In [ ]:
print(frame_length)
print(num_frames)
print(numpy.shape(frames))

WINDOW

In [ ]:
frames *= numpy.hamming(frame_length)
# frames *= 0.54 - 0.46 * numpy.cos((2 * numpy.pi * n) / (frame_length - 1))  # Explicit Implementation **

Fourier-Transform and Power Spectrum

In [ ]:
NFFT = 512
mag_frames = numpy.absolute(numpy.fft.rfft(frames, NFFT))  # Magnitude of the FFT
pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum

Filter Banks

In [ ]:
nfilt = 40
low_freq_mel = 0
high_freq_mel = (2595 * numpy.log10(1 + (sample_rate / 2) / 700))  # Convert Hz to Mel
mel_points = numpy.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
hz_points = (700 * (10**(mel_points / 2595) - 1))  # Convert Mel to Hz
bin = numpy.floor((NFFT + 1) * hz_points / sample_rate)

fbank = numpy.zeros((nfilt, int(numpy.floor(NFFT / 2 + 1))))
for m in range(1, nfilt + 1):
    f_m_minus = int(bin[m - 1])   # left
    f_m = int(bin[m])             # center
    f_m_plus = int(bin[m + 1])    # right

    for k in range(f_m_minus, f_m):
        fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
    for k in range(f_m, f_m_plus):
        fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
filter_banks = numpy.dot(pow_frames, fbank.T)
filter_banks = numpy.where(filter_banks == 0, numpy.finfo(float).eps, filter_banks)  # Numerical Stability
filter_banks = 20 * numpy.log10(filter_banks)  # dB

In [ ]:
plt.figure(figsize=(10, 6))
librosa.display.specshow(librosa.amplitude_to_db(filter_banks, ref=numpy.max),
                         sr=sample_rate, hop_length=512, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.title('Filterbank Spectrogram')
plt.show()

DCT dan Cepstral Liftering

In [ ]:
num_ceps = 12
mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, 1 : (num_ceps + 1)] # Keep 2-13

In [ ]:
cep_lifter = 22
(nframes, ncoeff) = mfcc.shape
n = numpy.arange(ncoeff)
lift = 1 + (cep_lifter / 2) * numpy.sin(numpy.pi * n / cep_lifter)
mfcc *= lift  #*

In [ ]:
plt.figure(figsize=(10, 4))
plt.imshow(mfcc, cmap='viridis', origin='lower', aspect='auto')
plt.title(f'MFCC')
plt.colorbar(format='%+2.0f dB')
plt.tight_layout()
plt.ylabel('Coefficients')
plt.xlabel('Frames')
plt.show()

Mean Normalization

In [ ]:
filter_banks -= (numpy.mean(filter_banks, axis=0) + 1e-8)
mfcc -= (numpy.mean(mfcc, axis=0) + 1e-8)

Visualization

In [ ]:
plt.figure(figsize=(10, 4))
plt.imshow(mfcc, cmap='viridis', origin='lower', aspect='auto')
plt.title(f'MFCC')
plt.colorbar(format='%+2.0f dB')
plt.tight_layout()
plt.ylabel('Coefficients')
plt.xlabel('Frames')
plt.show()